## Descripción del proyecto
Trabajas para la tienda online Ice que vende videojuegos por todo el mundo. Las reseñas de usuarios y expertos, los géneros, las plataformas (por ejemplo, Xbox o PlayStation) y los datos históricos sobre las ventas de juegos están disponibles en fuentes abiertas. Tienes que identificar patrones que determinen si un juego tiene éxito o no. Esto te permitirá detectar proyectos prometedores y planificar campañas publicitarias.
Delante de ti hay datos que se remontan a 2016. Imaginemos que es diciembre de 2016 y estás planeando una campaña para 2017.

Lo importante es adquirir experiencia de trabajo con datos. Realmente no importa si estás pronosticando las ventas de 2017 en función de los datos de 2016 o las ventas de 2027 en función de los datos de 2026.
El dataset contiene la abreviatura ESRB. The Entertainment Software Rating Board (la Junta de clasificación de software de entretenimiento) evalúa el contenido de un juego y asigna una clasificación de edad como Adolescente o Adulto.

**Importamos librerías**

In [30]:
import pandas as pd
import numpy as np
import scipy as sc
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.impute import KNNImputer


**Paso 1. Abre el archivo de datos y estudia la información general** 

In [2]:
df=pd.read_csv('data_frame/games.csv')
print(df.head(10))

                        Name Platform  Year_of_Release         Genre  \
0                 Wii Sports      Wii           2006.0        Sports   
1          Super Mario Bros.      NES           1985.0      Platform   
2             Mario Kart Wii      Wii           2008.0        Racing   
3          Wii Sports Resort      Wii           2009.0        Sports   
4   Pokemon Red/Pokemon Blue       GB           1996.0  Role-Playing   
5                     Tetris       GB           1989.0        Puzzle   
6      New Super Mario Bros.       DS           2006.0      Platform   
7                   Wii Play      Wii           2006.0          Misc   
8  New Super Mario Bros. Wii      Wii           2009.0      Platform   
9                  Duck Hunt      NES           1984.0       Shooter   

   NA_sales  EU_sales  JP_sales  Other_sales  Critic_Score User_Score Rating  
0     41.36     28.96      3.77         8.45          76.0          8      E  
1     29.08      3.58      6.81         0.77     

In [3]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16713 non-null  object 
 1   Platform         16715 non-null  object 
 2   Year_of_Release  16446 non-null  float64
 3   Genre            16713 non-null  object 
 4   NA_sales         16715 non-null  float64
 5   EU_sales         16715 non-null  float64
 6   JP_sales         16715 non-null  float64
 7   Other_sales      16715 non-null  float64
 8   Critic_Score     8137 non-null   float64
 9   User_Score       10014 non-null  object 
 10  Rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB
None


Paso 2. Prepara los datos

In [4]:
#Reemplazamos los nombres de las columnas en minusculas
df.columns=df.columns.str.lower()
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16713 non-null  object 
 1   platform         16715 non-null  object 
 2   year_of_release  16446 non-null  float64
 3   genre            16713 non-null  object 
 4   na_sales         16715 non-null  float64
 5   eu_sales         16715 non-null  float64
 6   jp_sales         16715 non-null  float64
 7   other_sales      16715 non-null  float64
 8   critic_score     8137 non-null   float64
 9   user_score       10014 non-null  object 
 10  rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB
None


In [5]:
#Vamos a cambiar la columna 'user_score' a numérico para hacer operaciones con estos valores, usamos el parámetro error=coerce para que no tenga en cuenta el objeto 'tbd'.
df.info()
df['user_score']=pd.to_numeric(df['user_score'],errors='coerce')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16713 non-null  object 
 1   platform         16715 non-null  object 
 2   year_of_release  16446 non-null  float64
 3   genre            16713 non-null  object 
 4   na_sales         16715 non-null  float64
 5   eu_sales         16715 non-null  float64
 6   jp_sales         16715 non-null  float64
 7   other_sales      16715 non-null  float64
 8   critic_score     8137 non-null   float64
 9   user_score       10014 non-null  object 
 10  rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16713 non-null  ob

Pasamos la columna 'user_score' a float, debido a que se trata de calificaciones numericas, por lo tanto debemos hacer operaciones con estos.  
Pasamos todos los tipos de datos incorrectos al correcto, ahora vamos a trabajar con los ausentes.

In [6]:
df['user_score']=df['user_score'].replace('tbd',np.nan)
print(df['user_score'].unique())

[8.  nan 8.3 8.5 6.6 8.4 8.6 7.7 6.3 7.4 8.2 9.  7.9 8.1 8.7 7.1 3.4 5.3
 4.8 3.2 8.9 6.4 7.8 7.5 2.6 7.2 9.2 7.  7.3 4.3 7.6 5.7 5.  9.1 6.5 8.8
 6.9 9.4 6.8 6.1 6.7 5.4 4.  4.9 4.5 9.3 6.2 4.2 6.  3.7 4.1 5.8 5.6 5.5
 4.4 4.6 5.9 3.9 3.1 2.9 5.2 3.3 4.7 5.1 3.5 2.5 1.9 3.  2.7 2.2 2.  9.5
 2.1 3.6 2.8 1.8 3.8 0.  1.6 9.6 2.4 1.7 1.1 0.3 1.5 0.7 1.2 2.3 0.5 1.3
 0.2 0.6 1.4 0.9 1.  9.7]


Consideramos los valores 'tbd' como valores ausentes, debido a que no son valores útiles para hacer calculos y sesgan más nuestros datos, por lo tanto los reemplazamos por valores nan.

In [7]:
print(100*df.isna().sum()/df.shape[0])
df.dropna(axis=0,inplace=True,subset=['name','genre'])
print(100*df.isna().sum()/df.shape[0])

name                0.011965
platform            0.000000
year_of_release     1.609333
genre               0.011965
na_sales            0.000000
eu_sales            0.000000
jp_sales            0.000000
other_sales         0.000000
critic_score       51.319174
user_score         54.591684
rating             40.478612
dtype: float64
name                0.000000
platform            0.000000
year_of_release     1.609526
genre               0.000000
na_sales            0.000000
eu_sales            0.000000
jp_sales            0.000000
other_sales         0.000000
critic_score       51.313349
user_score         54.586250
rating             40.471489
dtype: float64


 Borramos los ausentes de la columna name y genre, ya que no son significantes en el dataframe. Con respecto a los ausentes de las columnas **'critic_score', 'user_score' y 'rating'**, al ser el porcentaje tan alto, mayor al **30% de los datos** no es viable eliminarnos ni rellenarlos con algun estadístico, debido a que esto modificaría una gran parte de los datos afectando el análisis.

In [8]:
df['year_of_release'].fillna(df['year_of_release'].median(),inplace=True)
print(100*df.isna().sum()/df.shape[0])

name                0.000000
platform            0.000000
year_of_release     0.000000
genre               0.000000
na_sales            0.000000
eu_sales            0.000000
jp_sales            0.000000
other_sales         0.000000
critic_score       51.313349
user_score         54.586250
rating             40.471489
dtype: float64


Para el caso de el año de realización lo llenamos con la **mediana**.

In [9]:
print(df.duplicated().sum())

0


Verificamos si existen duplicados en nuestros datos.

In [10]:
#Calculamos el total de ventas de todas las regiones.
df['total_sales']=df['na_sales']+df['eu_sales']+df['jp_sales']
print(df.iloc[:,4:])

       na_sales  eu_sales  jp_sales  other_sales  critic_score  user_score  \
0         41.36     28.96      3.77         8.45          76.0         8.0   
1         29.08      3.58      6.81         0.77           NaN         NaN   
2         15.68     12.76      3.79         3.29          82.0         8.3   
3         15.61     10.93      3.28         2.95          80.0         8.0   
4         11.27      8.89     10.22         1.00           NaN         NaN   
...         ...       ...       ...          ...           ...         ...   
16710      0.00      0.00      0.01         0.00           NaN         NaN   
16711      0.00      0.01      0.00         0.00           NaN         NaN   
16712      0.00      0.00      0.01         0.00           NaN         NaN   
16713      0.01      0.00      0.00         0.00           NaN         NaN   
16714      0.00      0.00      0.01         0.00           NaN         NaN   

      rating  total_sales  
0          E        74.09  
1      

**Paso 3. Analiza los datos**

- **Juegos lanzados por año**

In [11]:
games_year=df.groupby('year_of_release',as_index=False)['name'].count()
fig=px.line(games_year,
            x='year_of_release',
            y='name',
            title='Juegos lanzados por año')
fig.show()

In [12]:
representative_df=df[df['year_of_release']>=1995]

Podemos ver que el gráfico tiene 2 puntos de inflexión importantes, a partír del año **1995** y el dato **máximo de ventas en 2007**, a partír de ahí las venras comienzan a bajar. Podemos decir que nuestros datos representantivos son **a partír del año 1995**, debido a que los números comienzan a aumentar.

- **Ventas por plataforma**

In [13]:
sales_platform=representative_df.pivot_table(index='platform',values='total_sales',aggfunc='sum')
sales_platform.sort_values(by='total_sales',ascending=False,inplace=True)
top_5_platforms=sales_platform.head(5)
top_5_platforms

,total_sales
platform,
PS2,1062.33
X360,885.66
Wii,828.44
PS3,803.97
DS,746.84


In [14]:
top_5_platforms_list=top_5_platforms.index
platform_distrib=representative_df[representative_df['platform'].isin(top_5_platforms_list)]
platform_distrib=platform_distrib.groupby(['year_of_release','platform'])['total_sales'].sum().reset_index()
platform_distrib.head()

,year_of_release,platform,total_sales
0,2000.0,PS2,35.59
1,2001.0,PS2,149.65
2,2002.0,PS2,183.66
3,2003.0,PS2,163.62
4,2004.0,DS,15.91


In [15]:
fig1=px.line(platform_distrib,x='year_of_release',y='total_sales',color='platform', title='Vida de cada consola')
fig1.show()

Analizando la vida de cada una de las consolas, estas crecen aproximadamente **5 años** hasta llegar a su punto máximo y vuelven y caen durante los siguientes **5 años**, por lo cual en general las antiguas y nuevas consolas tienen un tiempo de aparición y desaparición de **5 años**.

- Selección de los datos:
Vamos a usar los datos de **2011 en adelante** debido a que son los años relevantes en los cuales las plataformas permanecen actualmente.

In [44]:
bounded_data=platform_distrib[platform_distrib['year_of_release']>=2011.0]
bounded_data.reset_index(drop=True).head()

,year_of_release,platform,total_sales
0,2011.0,DS,24.30
1,2011.0,PS2,0.35
2,2011.0,PS3,135.22
3,2011.0,Wii,54.37
4,2011.0,X360,131.11


Vamos a análizar con diagramas de caja las ventas globales por plataforma teniendo en cuenta las plataformas más exitosas.

In [45]:
fig=px.box(bounded_data,x='platform',y='total_sales',title='Distribución de ventas por plataforma',color='platform')
fig.show()

In [48]:
for platform in top_5_platforms_list:
    prom=bounded_data[bounded_data['platform']==platform]['total_sales'].mean().round(2)
    print(f'El promedio de ventas de la plataforma {platform} es {prom} millones de dolares.')

El promedio de ventas de la plataforma PS2 es 0.35 millones de dolares.
El promedio de ventas de la plataforma X360 es 57.88 millones de dolares.
El promedio de ventas de la plataforma Wii es 14.52 millones de dolares.
El promedio de ventas de la plataforma PS3 es 63.46 millones de dolares.
El promedio de ventas de la plataforma DS es 12.05 millones de dolares.


Las diferencias en las ventas es el PS3 con un **75% de los datos debajo de 95.85 millones de dolares y una mediana de 65.875 millones de dolares**, seguido del X360 con un **75% de los datos debajo de 91.45 millones de dolares y una mediana de 56,215 millones de dolares.** Le sigue el Nintendo DS  y el Wii que tiene un valor atípico. Análizando los promedios y datos estadísticos, por el momento **la PS3 y la X360** tienen las mayores ventas en promedio y en su distribución de valores